## 13.4 sklearnのone-hotエンコーディングとトランスフォーマーのパイプライン
```sklearn```のパイプラインを用いた重回帰分析のスニペット。

```pandas.loc[]```を前提にしているため、pandasへの変換が必要。

### (共通)データの読み込み
```seaborn```の```tips```データセットを用いる。

In [ ]:
import polars as pl
import seaborn as sns

# データの読み込み
tips = pl.DataFrame(sns.load_dataset("tips"))
display(tips.head())

In [ ]:
from sklearn import linear_model

# 前処理関係のライブラリをインポートする
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import numpy as np

# ダミー変数化したい変数を指定する
categorical_features = ["sex", "smoker", "day", "time"]
categorical_transformer = OneHotEncoder(drop = "first")

# 前処理器の初期化
preprocessor = ColumnTransformer(
    transformers = [
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder = "passthrough"# 処理しない変数はそのまま通す
)

# パイプラインの初期化
pipe = Pipeline(
    steps = [
        ("preprocessor", preprocessor),
        ("lr", linear_model.LinearRegression())
    ]
)

# 学習
pipe.fit(
    X = tips[["total_bill", "size", "sex", "smoker", "day", "time"]].to_pandas(),
    y = tips["tip"].to_pandas(),
)

# 係数と切片を表示する
coefficients = np.append(
    pipe.named_steps["lr"].intercept_, pipe.named_steps["lr"].coef_
)
labels = np.append(
    ["intercept"], pipe[:-1].get_feature_names_out()
)
coefs = pl.DataFrame({"variables": labels, "coef": coefficients})
display(coefs)